In [ ]:
import pandas as pd
import numpy as np
from IPython import display
from matplotlib import image
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import ipywidgets as widgets
%matplotlib notebook
from ipywidgets import *
import matplotlib.pyplot as pltp
%matplotlib inline

In [ ]:
stationlocs = pd.read_csv("assets/BARTLocationPercentages.csv")
stationlocs.head(5)

In [ ]:
vizpic = display.Image("assets/BARTtracksmap.png")
picdata = image.imread('assets/BARTtracksmap.png')

In [ ]:
testlog = pd.read_csv("occ-log-classification/csv/labeled.csv").head(60)
testlog.head(5)

In [ ]:
station_list = (stationlocs['Station_Name']).tolist()
s_abbrv = [station[:3] for station in station_list]

In [ ]:
def lookfornames(c):
    for s in s_abbrv:
        if s in c:
            return s
        
stationlocs['Location'] = testlog["Log"].apply(lookfornames)
testlog['Location'] = testlog["Log"].apply(lookfornames)

In [ ]:
stationlocs.head(10)

In [ ]:
justxandy = stationlocs.drop(columns=['Station_Name', 'Rain_Critical', 'Asset_Location'])
justxandy.head(5)

In [ ]:
merged = testlog.merge(justxandy.set_index('Location'), on='Location')
merged.head(5)

In [ ]:
merged['dupped'] = merged.duplicated(subset=['Log'])
merged = merged[merged.dupped != True]

In [ ]:
vizdata = merged.drop(columns=['dupped'])

In [ ]:
from tensorflow import keras
model = keras.models.load_model('occ-log-classification/model')

In [ ]:
class_legend = {
    0:"Misc",
    1:'Other BPD',
    2:'Homeless',
    3:'Medical',
    4:'Patron interference',
    5:"Vehicle failure",
    6:"Wayside equipment",
    7:"Software related failures",
    8:"Human Error",
    9:"Weather",
    10:"Info (no error)",
    11:"Delays",
    12:"Track obstruction",
    13:"Schedule maintenance"
}

In [ ]:
vizdata['Cat_Pred'] = vizdata['Log'].apply(lambda log: np.argmax(model.predict([log]), axis=1)[0])

In [ ]:
vizdata['Class_Pred'] = vizdata['Cat_Pred'].apply(lambda code: class_legend[code])

In [ ]:
vizdata.head(5)

In [ ]:
def update(time = 622):
  
    fig = plt.figure(figsize=(10, 8), dpi=200)
    plt.imshow(picdata)
    for index,row in vizdata.iterrows():
        if row['Time'] <= time:
            plt.plot(1200* (row['X_Percentage']/100), 1100 * (row['Y_Percentage']/100), marker='v', color="blue")

    fig.canvas.draw_idle()

interact(update, time = widgets.IntSlider(value=622, min=500, max=1200, step=1));

In [ ]:
maptable = vizdata
maptable["xplot"] = (1200* (maptable['X_Percentage']/100))
maptable["yplot"] = (1080* (maptable['Y_Percentage']/100))
maptable
frametable = maptable[["xplot", "yplot"]]
xvals = maptable['xplot']
yvals = maptable['yplot']
maptable.head(5)

In [ ]:
#plotly conversion hopefully
import plotly.graph_objects as go
import base64
import plotly.express as px
# Create figure
def plotlyconvgeo():
    mapfig = go.Figure()
    img_width = 1200
    img_height = 1080
    mapfig.update_layout(
        autosize=False
    )

    bartimage = base64.b64encode(open("assets/BARTtracksmap.png", 'rb').read())
    mapfig.add_layout_image(
            dict(
                source='data:image/png;base64,{}'.format(bartimage.decode()), #"./assets/BARTtracksmap.png",
                xref="x",
                yref="y",
                x=0,
                y=0,
                sizex=img_width,
                sizey=img_height,
                #sizing="stretch",
                opacity=1,
                sizing='contain',
                layer="below",xanchor="left", yanchor="top")
    )
    mapfig.update_xaxes(showgrid=False, range=(0, img_width))
    mapfig.update_yaxes(showgrid=False, scaleanchor='x', range=(img_height, 0))

    mapfig.add_trace(
        go.Scatter(x=maptable["xplot"], y=maptable["yplot"], mode="markers", text=maptable['Log'])

    )

    #mapfig.show()
    return mapfig
plotlyconvgeo()

In [ ]:
catvizdata = vizdata.drop(columns=['X_Percentage', 'Y_Percentage'])
refs = []
refs = np.random.choice(np.arange(len(catvizdata['Location'])), len(catvizdata['Location']), replace=False)

catvizdata['Reference'] = refs
catvizdata.head(10)

In [ ]:
import datetime
timarr = []
for t in catvizdata['Time']:
    t = int(t)
    temp = datetime.time(t//100, t%100)
    timarr.append(temp)
catvizdata['Time'] = timarr

In [ ]:
from datetime import time
timeaxis = []
start = datetime.time(5, 0, 0)
end = datetime.time(22, 0, 0)
currhr = 5
currmin = 1
timeaxis.append(start)
for i in range(1020):
    nexttime = datetime.time(currhr, currmin, 0)
    timeaxis.append(nexttime)
    if currmin == 59:
        currmin = 0
        currhr = currhr + 1
    else:
        currmin = currmin + 1
catvizdata = catvizdata.sort_values(by=['Time'])
xaxistime = catvizdata['Time']

In [ ]:
import plotly.express as px
swimscatter = px.scatter(catvizdata, y="Class_Pred", x='Time', color="Class_Pred", symbol="Class_Pred", hover_data=['Log', 'Location'])
swimscatter.update_traces(marker_size=20)
swimscatter.update_xaxes(categoryorder='category ascending', showgrid=False)
swimscatter.show()

In [ ]:
# import base64
# import datetime
# import io
# import dash

# from dash.dependencies import Input, Output, State
# from dash import dcc, html, dash_table

# import pandas as pd

# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# app.layout = html.Div([
#     dcc.Upload(
#         id='upload-data',
#         children=html.Div([
#             'Drag and Drop or ',
#             html.A('Select Files')
#         ]),
#         style={
#             'width': '100%',
#             'height': '60px',
#             'lineHeight': '60px',
#             'borderWidth': '1px',
#             'borderStyle': 'dashed',
#             'borderRadius': '5px',
#             'textAlign': 'center',
#             'margin': '10px'
#         },
#         # Allow multiple files to be uploaded
#         multiple=True
#     ),
#     html.Div(id='output-data-upload'),
# ])

# def parse_contents(contents, filename, date):
#     content_type, content_string = contents.split(',')
#     decoded = base64.b64decode(content_string)
#     df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
#     return geoplotly(df)

# @app.callback(Output('output-data-upload', 'children'),
#               Input('upload-data', 'contents'),
#               State('upload-data', 'filename'),
#               State('upload-data', 'last_modified'))

# def update_output(list_of_contents, list_of_names, list_of_dates):
#     if list_of_contents is not None:
#         children = [
#             parse_contents(c, n, d) for c, n, d in
#             zip(list_of_contents, list_of_names, list_of_dates)]
#         return children

# if __name__ == '__main__':
#     app.run_server(debug=True, use_reloader=False)